# Reading the data file using python 

### all the imports

In [76]:
import os


## getting all the files from the directory

In [2]:
def get_files():
    all_file_path = []
    """walk through the data dir to its sub-dirs and gets all the file with .review extension and return all the file_paths list"""
    # looping through the directory and the sub-directory
    for root, dirs, files in os.walk("data"):
        for file in files:
            # getting the file end with .review
            if file.endswith(".review"):
                # joining the file path with the actual full path of the file 
                file_path = os.path.join(root, file)
                # appending the file_path in the list 
                all_file_path.append(file_path)
    return all_file_path


## categorized the files into list


In [3]:
def categorise_files(all_file_path):
    """get all file path and categorised the files into positive, negative and unlabeled review and return the file path list as tuple"""
    
    pos_review_f_path = []
    neg_review_f_path=[]
    unl_review_f_path = []
    
    for file in all_file_path:
        if "positive" in file:
            pos_review_f_path.append(file)
        elif "negative" in file:
            neg_review_f_path.append(file)
        else:
            unl_review_f_path.append(file)
    return pos_review_f_path, neg_review_f_path, unl_review_f_path

## reading the files

In [84]:
def read_file(file_path):
    """reads the file from file path and return all lines in list"""
    with open(file_path) as file:
        items = file.readlines()
    return items

## making the list of review after reading the file

In [86]:
def build_review_list(items):
    """gets all the items, prepare a list of each review and returns the list of reviews"""
    reviews = []
    review = []
    for item in items:
        # remove \n from the item
        item = item[:-1]
        # get the file inside the review block not including the start and the end block
        if item != '<review>' and item != '</review':
            review.append(item)
            
        #  adding the review to the review list  
        if item == '</review>':
            reviews.append(review)
            review = []
    return reviews
   

In [88]:

# we need to run read_file methods to all of the file to get the data 
# list to store positive review
# all_positive_reviews = []
# for file in positive_review_file_list:
#     all_positive_reviews.append(read_file(file))
# len(reviews)

## processing the review and making it ready to transfer into the dataframe

In [89]:
def process_review_list(reviews):
    """gets the list of reviews, seperate the headings and values and returns as list of dictionary"""
    dict_review_list = []
    previous_key = None
    
    #iterating through reviews
    for review in (reviews):
        dict_review = {}
        #iterating through a single review
        for num, item in enumerate(review):
            #eliminating null values
            if item != "":
                #getting and heading and assigning with corresponding values
                if item[0] =="<" and item[1] != "/":
                    dict_review[item] = review[num + 1]
                    previous_key = item
                #checking if any values are missing and reassigning them
                if item not in dict_review.values() and item[0] != '<':
                    dict_review[previous_key] += item
        # when a review is ready adding it to the list
        dict_review_list.append(dict_review)
    return dict_review_list